Secure & Distributed Data Sharing with Blockchain and Shamir’s Secret Sharing

Project is Divided into 2 parts
- Encryption 
- Decryption

1- Encryption

In [1]:
#Libraries needed
from cryptography.fernet import Fernet
from secrets import token_bytes
from datetime import datetime, timedelta  # for timestamps
import os  # for folder creation
import random  # for simulating temperature readings
import json  # for transaction data
import shamirs

In [2]:
#Blockchain simulation
class Block:
    """Simulates a blockchain block"""
    def __init__(self, index, timestamp, data, previous_hash):
        self.index = index
        self.timestamp = timestamp
        self.data = data
        self.previous_hash = previous_hash
        self.hash = self.calculate_hash()

    def calculate_hash(self):
        # Simplified hash calculation (replace with a real hashing algorithm)
        data_string = json.dumps(self.__dict__, sort_keys=True)
        return hash(data_string)

class Blockchain:
    """Simulates a simple blockchain"""
    def __init__(self):
        self.chain = []
        self.create_genesis_block()

    def create_genesis_block(self):
        """Creates the first block (genesis block)"""
        genesis_data = {"message": "Honeycomb Demo Blockchain"}
        self.chain.append(Block(0, datetime.utcnow(), genesis_data, "0"))

    def get_latest_block(self):
        return self.chain[-1]

    def add_block(self, block):
        self.chain.append(block)

    def proof_of_work(self, block):
        """Simplistic Proof of Work (replace with a real consensus mechanism)"""
        while True:
            block.hash = block.calculate_hash()
            if block.hash[:2] == "00":  # Adjust difficulty as needed
                break

Getting Data as input in the form of fragments

In [3]:
def get_temperature():
    """Simulates a temperature sensor reading"""
    return random.randint(18, 28)  # Adjust range as needed

def create_data_fragment(num_readings, time_interval):
    """Creates a data fragment with multiple temperature readings"""
    readings = []
    current_time = datetime.now()
    for _ in range(num_readings):
        temperature = get_temperature()
        reading_time = current_time.strftime("%H:%M:%S")  # Format timestamp
        readings.append(f"{reading_time}: {temperature}°C")
        current_time += timedelta(minutes=time_interval)  # Simulate time interval

    data_fragment = "\n".join(readings)
    return data_fragment

Data encryption and key generation

In [4]:
def encrypt_data(data):
    # Generate encryption key
    key = Fernet.generate_key()
    fernet = Fernet(key)

    # Encrypt data fragment
    encrypted_data = fernet.encrypt(data.encode())
    return encrypted_data, key


KEY shares and storing all shares

In [5]:
def split_key(key, num_shares, threshold):
    # Key sharding
    shares = shamirs.shares(key, num_shares, threshold)
    return shares

def store_shares(shares, folder_paths):
    for i, share in enumerate(shares):
        # Prepend random data for obfuscation (optional)
        share_data = token_bytes() + share
        
        # Create folder if it doesn't exist
        if not os.path.exists(folder_paths[i]):
            os.mkdir(folder_paths[i])

        # Store share in a file
        with open(f"{folder_paths[i]}/share_{i+1}.txt", "wb") as f:
            f.write(share_data)

Creating block chain transaction

In [6]:
def create_transaction(data_fragment_id, encrypted_data, share, folder_location):
    """Creates a transaction object (conceptual)"""
    transaction = {
        "data_fragment_id": data_fragment_id,
        "encrypted_data": encrypted_data.decode(),  # Assuming string format for simulation
        "key_share": share.decode(),  # Assuming string format for simulation
        "folder_location": folder_location
    }
    return transaction

In [ ]:
blockchain = Blockchain()
# Define number of readings and time interval for fragment
num_readings = 4  # Number of temperature readings in a fragment
time_interval = 5  # Time interval between readings (minutes)

# Simulate generating multiple data fragments (replace with actual data source)
data_fragments = []
for i in range(2):  # Simulate 2 data fragments
    data_fragment = create_data_fragment(num_readings, time_interval)
    data_fragments.append(data_fragment)

# List to store transactions for key shares
transactions = []

# Process each data fragment
for i, data_fragment in enumerate(data_fragments):
    # Encryption and key sharding
    encrypted_data, key = encrypt_data(data_fragment)
    num_shares = 3  # Number of key shares
    threshold = 2  # Minimum shares needed for reconstruction
    shares = split_key(key, num_shares, threshold)
    # Store encrypted data (simulation)
    chosen_folder = f"folder{i+1}"  # Simulate separate folders for each fragment
    with open(f"{chosen_folder}/data.txt", "wb") as f:
        f.write(encrypted_data)

    # Simulate blockchain storage for key shares
    folder_paths = [f"folder{j+1}" for j in range(3)]  # Match folder structure
    for j, share in enumerate(shares):
        # Create transaction data
        transaction_data = create_transaction(
            data_fragment_id=f"temperature_sensor_readings_{i+1}",
            share=share,
            folder_location=folder_paths[j]
        )
        transactions.append(transaction_data)

for transaction in transactions:
    print("Simulating sending transaction to blockchain network:", transaction)

print("Data fragments encrypted and key shares stored (simulated)!")

2- Decryption

In [8]:
def decrypt_data_fragment(encrypted_data, key):
  """Decrypts the data fragment using the reconstructed key"""
  fernet = Fernet(key)
  decrypted_data = fernet.decrypt(encrypted_data).decode()  # Assuming string format
  return decrypted_data

Retrieve Data

In [9]:
def retrieve_data(data_fragment_identifier, blockchain_data):
  """Retrieves and decrypts the data fragment based on blockchain information"""
  folder_paths = []
  for transaction in blockchain_data:
    if transaction["data_fragment_identifier"] == data_fragment_identifier:
      # Assuming "folder_location" key exists in transaction data
      folder_paths.append(transaction["folder_location"])

  # Check if all key share locations are retrieved
  if len(folder_paths) != 3:
    print("Error: Insufficient information retrieved from blockchain!")
    return None

  shares = []
  for folder_path in folder_paths:
    # Read share data from file (assuming appropriate storage logic)
    with open(f"{folder_path}/share.txt", "rb") as f:
      share_data = f.read()
    # Remove prepended random data (optional)
    share = share_data[len(token_bytes()):]
    shares.append(share)

  # Reconstruct key from shares
  try:
    key = Shamir.combine(shares, 3)  # Threshold of 3 for all shares
  except Shamir.exceptions.ShareError:
    print("Error: Key reconstruction failed!")
    return None

  # Retrieve encrypted data (replace with actual storage logic)
  chosen_folder = f"folder1"  # Adapt based on data retrieval logic (e.g., transaction data)
  with open(f"{chosen_folder}/data.txt", "rb") as f:
    encrypted_data = f.read()

  # Decrypt data fragment
  decrypted_data = decrypt_data_fragment(encrypted_data, key)
  return decrypted_data

In [ ]:
data_fragment_identifier = "temperature_sensor_readings_1"

blockchain_data = [
  {
    "data_fragment_identifier": "temperature_sensor_readings_1",
    "folder_location": "folder1/share.txt"
  },
  {
    "data_fragment_identifier": "temperature_sensor_readings_1",
    "folder_location": "folder2/share.txt"
  },
  {
    "data_fragment_identifier": "temperature_sensor_readings_1",
    "folder_location": "folder3/share.txt"
  }
]

decrypted_data = retrieve_data(data_fragment_identifier, blockchain_data)

if decrypted_data:
  print(f"Retrieved and decrypted data for {data_fragment_identifier}:")
  print(decrypted_data)
else:
  print("Failed to retrieve data!")
